# CS336 Language Modeling from Scratch 从零开始构建语言模型

## 1. 关于这门课程

### 1.1 课程简介: 
- 语言模型开启了一个全新的范式————通过一个通用系统来解决各种**下游任务**。本课程将介绍语言模型创建的各个环节————数据收集与清洗、Transformer 模型的构建、模型训练，以及部署前的评估。
- 开设这门课的原因：科研人员正逐渐与底层技术脱节（自己实现模型$\rightarrow$对现有模型微调$\rightarrow$写提示词），但是基础研究必须深入了解底层技术才能完成。本课程的目标就是通过**搭建**语言模型来深入理其深层原理。这门课会培养工程能力，注重实现与优化。
  > **下游任务**：
  > 指利用已经训练好的语言模型在具体应用场景中执行的各种自然语言处理任务。例如文本分类、命名实体识别（NER，例如识别人名、地名、组织名等）、问答系统（如基于知识库或文档的问答）、文本生成（如对话生成、文章续写、代码生成、文本摘要（生成文章或文档的简要版本）、机器翻译（如英文翻译成中文）、语义相似度判断（判断两段文本是否语义相近）、文本纠错与语法改进等
- 先决条件：python，深度学习与系统优化，微积分与线性代数，概率与统计，机器学习基础
- 课程内容及作业安排。课程一共包含5个部分：basics、systems、scaling laws、data、alignment。每个部分对应一次作业，没有代码框架（scaffolding），但会提供单元测试与接口适配器帮助你检查功能是否正确。在本地实现和测试，通过后再提交至集群运行以进行准确率与速度评估。部分作业会设有排行榜。**尽量不使用AI编程工具**。
![design](images\design-decisions.png)


### 1.2 模型更大的时候会有所不同
前沿的模型对我们来说遥不可及，本课程中构建的小语言模型无法完全代表大预言模型的真实特征。
> **直觉**（intuitions）：有些设计决策目前**根本无法（或尚且无法）通过理论充分解释**，往往只能靠反复实验得出。
- 例1：随着模型规模扩大，注意力机制在计算中所占的比例迅速减少，而前馈层变成了真正的主力计算负担。
  ![](images\roller-flops.png)
  | 列名                | 含义                                  |
  | ----------------| ---------------------------------------- |
  | **description** | 模型的规模（参数数量）                     |
  | **FLOPs/update**| 每次参数更新所需的总浮点运算次数（FLOPs）    |
  | **FLOPS MHA**   | 多头注意力（Multi-Head Attention）部分占总 FLOPs 的百分比 |
  | **FLOPS FFN**   | 前馈神经网络（Feed-Forward Network）部分的 FLOPs 占比     |
  | **FLOPS attn**  | 计算注意力权重（attention scores）的 FLOPs 占比           |
  | **FLOPS logit** | 输出 logits（即分类器部分）的 FLOPs 占比                  |
- 例2：随着**模型规模**的增加，模型会在某些任务上突然出现**能力涌现（emergent abilities）**。
  ![](images\wei-emergence-plot.png)


### 1.3 这门课上能学到的能够迁移到前沿模型的知识
- 运行机制：Transformer原理、模型并行利用GPU
- 思维方式：如何最大限度地榨干硬件性能、认真对待规模问题（例如规模法则scaling laws）
- 直觉经验：在数据选择和建模决策中，哪些因素有助于提高准确率
机制和思维方式是可以迁移到大型模型上的，而直觉经验在不同规模下并不总是适用的。
> **Scaling laws**（规模法则）：通过大量实验观察，随着模型的规模、训练数据量和计算量的增长，模型性能系统性地提升的经验规律。[Scaling Laws for Neural Language Models](https://arxiv.org/pdf/2001.08361)
>| 增加的因素| 对性能的影响（正面） | 影响机制说明 | 注意事项 |
>| ------------- | ---------- | ---------- | ---------- |
>| **参数量** | 提升模型表达能力 | 更大的模型可以学习更复杂的模式，有能力拟合更大的语料和上下文信息| 如果数据量不足，可能导致过拟合。超大模型计算开销显著增加|
>| **数据量** | 提升模型泛化能力 | 更多样本可以帮助模型学习更全面的语言规律，降低训练偏差与过拟合风险|如果模型太小，无法充分利用大量数据（参数成为瓶颈）|
>| **计算量** | 提升整体训练质量 | 可以训练更久，达到更低损失。支持更大 batch size、更长训练周期、更多训练 steps。也代表更大的硬件资源投入 | 计算资源要与数据量/模型规模相匹配，不然会“浪费”预算（比如训练很小的模型却用超多计算） |


### 1.4 效率驱动设计
**错误**的理解：只有规模重要，算法不重要。
**正确**的理解：能**扩展**（scale）的**算法**才是关键。在大规模下效率更为重要，因为资源消耗巨大，不能浪费。
$$ \text{accuracy} = \text{efficiency} \times \text{resources} $$
思考方式的转变：在给定的计算资源和数据预算下，你能构建的最优模型是什么？核心目标是：**最大化效率**，即在固定资源条件下，如何训练出最优的模型？
$$ \text{resourse} = \text{data} + \text{hardware} $$
目前，我们受限于计算资源，因此必须最大限度地利用现有硬件性能：
- 数据处理（Data processing）：避免在低质量或无关数据上浪费宝贵的计算资源。
- 分词（Tokenization）：虽然直接处理原始字节非常优雅，但对当前模型架构而言，这样做计算效率极低。
- 模型架构（Model architecture）：很多架构上的变动，都是为了减少内存消耗或降低 FLOPs，例如：共享 KV 缓存、滑动窗口注意力等。
- 训练（Training）：在优化策略得当的情况下，**只进行一次完整训练（single epoch）**也可能足够。
- 扩展法则（Scaling laws）：使用较小模型进行超参数调优，从而节省计算资源。
- 对齐（Alignment）：如果模型能更好地对齐于具体用途，那么所需的基础模型可以更小。
在不远的未来，我们将会面临**数据受限**（data-constrained）的局面


## 2. 语言模型的发展状况
### 2.1 发展历程
1. 神经网络之前（2010年之前）
- 使用语言模型来衡量英文的熵[Shannon 1950]
- 大量基于 n-gram 的语言模型研究（应用于机器翻译、语音识别等）[Brants 等，2007]
 
2. 神经网络成分出现（2010 年代）
- 第一种神经语言模型[Bengio 等，2003]
- 序列到序列建模（用于机器翻译）[Sutskever 等，2014]
- Adam 优化器[Kingma 等，2014]
- 注意力机制（用于机器翻译）[Bahdanau 等，2014]
- Transformer 架构（用于机器翻译）[Vaswani 等，2017]
- Mixture of Experts（专家混合模型）[Shazeer 等，2017]
- 模型并行技术[Huang 等，2018]，[Rajbhandari 等，2019]，[Shoeybi 等，2019]
 
3. 早期基础模型（2010 年代末）
- ELMo：基于 LSTM 的预训练，微调能显著提升性能[Peters 等，2018]
- BERT：基于 Transformer 的预训练，广泛提升多任务性能[Devlin 等，2018]
- Google T5（110 亿参数）：将所有任务统一为文本到文本的格式[Raffel 等，2019]
 
4. 拥抱扩展性，走向闭源
- OpenAI GPT-2（15 亿参数）：生成流畅文本，首次展示 zero-shot 能力，采用分阶段发布策略[Radford 等，2019]
- Scaling laws（扩展法则）：提供规模扩展的可预测性与希望[Kaplan 等，2020]
- OpenAI GPT-3（175 亿参数）：引入上下文学习（in-context learning），完全闭源[Brown 等，2020]
- Google PaLM（5400 亿参数）：超大规模，但训练轮数不足[Chowdhery 等，2022]
- DeepMind Chinchilla（700 亿参数）：遵循计算最优的 scaling laws[Hoffmann 等，2022]
  > **闭源模型**（如 GPT-4o）：仅通过 API 提供访问[OpenAI，2023]
  > **开放权重模型**（如 DeepSeek）：开放模型权重与结构细节，但不公开数据[DeepSeek-AI，2024]
  > **开源模型**（如 OLMo）：开放模型权重与训练数据，并提供较完整论文（可能缺乏失败实验与设计动机）[Groeneveld，2024] 

5. 开源模型崛起
- EleutherAI 提供开放数据集（The Pile）和模型（如 GPT-J）[Gao 等，2020] [Wang 等，2021]
- Meta OPT（175B）：尝试复现 GPT-3，过程中遇到大量硬件问题[Zhang 等，2022]
- Hugging Face / BigScience 的 BLOOM：注重数据来源与开放性[Workshop等，2022]
- Meta 的 Llama 系列[Touvron 等，2023] [Grattafiori 等，2024]
- 阿里巴巴的 Qwen 模型系列[Qwen 等，2024]
- DeepSeek 系列模型[DeepSeek-AI 等，2024]
- AI2 的 OLMo 2 模型[Groeneveld 等，2024] [OLMo 项目，2024]

6. 当今前沿模型（2025 年）

- [OpenAI o3](https://openai.com/index/openai-o3-mini/)，[Anthropic Claude Sonnet 3.7](https://www.anthropic.com/news/claude-3-7-sonnet)，[xAI Grok 3](https://x.ai/news/grok-3)，[Google Gemini 2.5](https://blog.google/technology/google-deepmind/gemini-model-thinking-updates-march-2025/)，[Meta Llama 3.3](https://ai.meta.com/blog/meta-llama-3/)，[DeepSeek r1](https://www.deepseek.com/)，[Alibaba Qwen 2.5 Max](https://qwenlm.github.io/blog/qwen2.5-max/)，[Tencent Hunyuan-T1](https://tencent.github.io/llm.hunyuan.T1/README_EN.html)


### 2.2 工业界的语言模型
GPT-4 拥有 1.8 万亿个参数，训练成本高达 1 亿美金；xAI 使用 20 万张 H100 GPU 组建集群，用于训练 Grok 模型；Stargate 项目计划在 4 年内投资 5000 亿美元。
> [星际之门计划（Stargate Project）](https://openai.com/index/announcing-the-stargate-project/)是美国一家人工智能合资企业，由OpenAI 、软银、甲骨文和投资公司 MGX 联合创立。该合资公司计划到2029年在美国的人工智能基础设施上投资高达**5000亿美元**。这一计划于2022年启动，并在2025年1月21日由美国总统唐纳德·川普正式宣布。该公司在美国特拉华州注册成立，名称为Stargate LLC。软银首席执行官孙正义被任命为公司董事长。
